# Regional economic dynamics, nighttime lights, and land cover: Insights from dimly lit islands

## Install Packages

In [1]:
import pandas as pd
# import geopandas as gpd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from pandasgui import show

## Preliminaries

In [2]:
# Set path
path = "/Users/Jesson Pagaduan/Google Drive/Geospatial_Project/"

## Batangas GDP by city/municipality from NTL

In [44]:
df_ntl_city = pd.read_excel(path + "data/scratch/NTL_2000-2018_PHL_municipal.xlsx", index_col=False,
                            usecols=['YEAR', 'GID_1', 'NAME_1', 'NAME_2', 'COUNT', 'AREA', 'SUM'], 
                            parse_dates=['YEAR'])
df_ntl_city.columns = [i.lower() for i in df_ntl_city.columns]

In [45]:
df_ntl_city.columns = ['year', 'prov_id', 'province', 'city', 'city_count', 'city_area', 'city_ntl']

In [46]:
df_ntl_city

,year,prov_id,province,city,city_count,city_area,city_ntl
0,2000-01-01,PHL.1_1,Abra,Bangued,521,0.010511,0.00
1,2000-01-01,PHL.1_1,Abra,Boliney,783,0.015796,0.00
2,2000-01-01,PHL.1_1,Abra,Bucay,436,0.008796,0.00
3,2000-01-01,PHL.1_1,Abra,Bucloc,214,0.004317,0.00
4,2000-01-01,PHL.1_1,Abra,Daguioman,412,0.008312,0.00
...,...,...,...,...,...,...,...
31288,2018-01-01,PHL.81_1,Zamboanga Sibugay,Roseller Lim,1109,0.022373,13.52
31289,2018-01-01,PHL.81_1,Zamboanga Sibugay,Siay,759,0.015312,8.41
31290,2018-01-01,PHL.81_1,Zamboanga Sibugay,Talusan,206,0.004156,0.00
31291,2018-01-01,PHL.81_1,Zamboanga Sibugay,Titay,720,0.014525,5.15


In [47]:
df_city_gdp = df_ntl_city.set_index(['province', 'year']).join(pd.read_excel('province-gdp.xlsx').set_index(['province', 'year']), rsuffix='_r')

In [33]:
show(df_city_gdp)

In [48]:
df_city_gdp = df_city_gdp.reset_index().set_index(['city', 'year'])

In [49]:
df_city_gdp = df_city_gdp.assign(city_ntl_share = lambda x: x.city_ntl / x.prov_ntl,
                                 log_ind_p1_c = lambda x: x.log_ind_p1 * x.city_ntl_share,
                                 log_ind_p2_c = lambda x: x.log_ind_p2 * x.city_ntl_share,
                                 log_svc_p1_c = lambda x: x.log_svc_p1 * x.city_ntl_share,
                                 log_svc_p2_c = lambda x: x.log_svc_p2 * x.city_ntl_share,
                                 log_non_agr_p1_c = lambda x: x.log_non_agr_p1 * x.city_ntl_share,
                                 log_non_agr_p2_c = lambda x: x.log_non_agr_p2 * x.city_ntl_share)

In [50]:
df_city_gdp.reset_index().to_excel('city-gdp.xlsx', index=False)

In [51]:
df_city_gdp_gr = df_city_gdp[['city_count', 'city_ntl', 'city_ntl_share',
                              'log_ind_p1_c', 'log_ind_p2_c', 'log_svc_p1_c', 'log_svc_p2_c',
                              'log_non_agr_p1_c', 'log_non_agr_p2_c']].diff() * 100

In [52]:
df_city_gdp_gr = df_city_gdp.join(df_city_gdp_gr, rsuffix='_gr')

In [54]:
df_city_gdp_gr.reset_index().to_excel('city-gdp-gr.xlsx', index=False)